## **DATASCIENCENIGERIA: STAFF PROMOTION PREDICTION**
*                                                             By Steven Kolawole

In [1]:
import pandas as pd
X = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

#dropping any row without target and separating target from predictors
X.dropna(axis=0, subset=['Promoted_or_Not'], inplace=True)

X.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


**Data Cleaning and EDA**

In [2]:
#Getting the total number of rows
print("The total number of rows in X: ", X.shape[0])
print("The total number of rows in X_test: ", X_test.shape[0])

#getting the columns with missing values
missing_values = pd.concat([X.isnull().sum(), X_test.isnull().sum()], 
                           keys=['X', 'X_test'], axis=1, sort=True)
print(missing_values)

The total number of rows in X:  38312
The total number of rows in X_test:  16496
                                        X  X_test
Channel_of_Recruitment                  0     0.0
Division                                0     0.0
EmployeeNo                              0     0.0
Foreign_schooled                        0     0.0
Gender                                  0     0.0
Last_performance_score                  0     0.0
Marital_Status                          0     0.0
No_of_previous_employers                0     0.0
Past_Disciplinary_Action                0     0.0
Previous_Award                          0     0.0
Previous_IntraDepartmental_Movement     0     0.0
Promoted_or_Not                         0     NaN
Qualification                        1679   730.0
State_Of_Origin                         0     0.0
Targets_met                             0     0.0
Training_score_average                  0     0.0
Trainings_Attended                      0     0.0
Year_of_birth      

In [3]:
#getting the value counts of the column with missing values
X['Qualification'].value_counts()

First Degree or HND         25578
MSc, MBA and PhD            10469
Non-University Education      586
Name: Qualification, dtype: int64

In [4]:
map_dict = {'Qualification': {
    'Non-University Education': 1, 
    'First Degree or HND': 2,
    'MSc, MBA and PhD': 3
}}

X = X.replace(map_dict)
X_test = X_test.replace(map_dict)

In [5]:
X.dtypes

EmployeeNo                              object
Division                                object
Qualification                          float64
Gender                                  object
Channel_of_Recruitment                  object
Trainings_Attended                       int64
Year_of_birth                            int64
Last_performance_score                 float64
Year_of_recruitment                      int64
Targets_met                              int64
Previous_Award                           int64
Training_score_average                   int64
State_Of_Origin                         object
Foreign_schooled                        object
Marital_Status                          object
Past_Disciplinary_Action                object
Previous_IntraDepartmental_Movement     object
No_of_previous_employers                object
Promoted_or_Not                          int64
dtype: object

In [6]:
X.groupby('Promoted_or_Not').mean()

,Qualification,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average
Promoted_or_Not,,,,,,,,
0,2.266565,2.258191,1986.167517,7.562730,2013.123777,0.320379,0.013943,54.641584
1,2.303894,2.204875,1986.661833,9.173095,2013.311941,0.705955,0.122802,63.210429


In [7]:
#Features based on the data provided by 'Promoted_or_Not'
features = ['Division', 'Training_score_average', 'Last_performance_score',
            'Qualification', 'Targets_met', 'Previous_Award', 'Trainings_Attended']

#separating the target from the predictors.
y = X['Promoted_or_Not']
X.drop(['Promoted_or_Not'], axis=1, inplace=True)

In [8]:
#Getting the numeric datatype columns
numerical_cols = [col for col in X.columns if col in features and X[col].dtype in ['float64', 'int64']]

#getting the columns with categorical data
categorical_cols = [col for col in X.columns if col in features and X[col].dtype == 'object']

#getting the number of unique entries in each categorical column
num_of_uniques = list(map(lambda col: X[col].nunique(), categorical_cols))
d = dict(zip(categorical_cols, num_of_uniques))
sorted(d.items(), key = lambda x: x[1])

[('Division', 9)]

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#preprocessing the numerical values
numerical_transformer = SimpleImputer(strategy='median')

#preprocessing the categorical values
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#bundling the numerical and categorical values together for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [10]:
from xgboost import XGBClassifier

#declaring the model
model = XGBClassifier(n_estimators=1000, learning_rate=0.07)

#bundling the preprocessor and model
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

In [11]:
from sklearn.model_selection import train_test_split

#splitting the training data for model validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    train_size=0.8, test_size=0.2, random_state=0)

In [12]:
from sklearn.metrics import accuracy_score

#fitting the model and using it to predict X_test
clf.fit(X_train, y_train)
pred_test = clf.predict(X_valid)

print("The model accuracy using f1_score: ", accuracy_score(y_valid, pred_test))

The model accuracy using f1_score:  0.9466266475270781


In [13]:
clf.fit(X, y)
predictions = clf.predict(X_test)

In [14]:
output = pd.DataFrame()
output['EmployeeNo'] = X_test['EmployeeNo']
output['Promoted_or_Not'] = predictions

output.to_csv('submission_local.csv', index=False)
output.head()

,EmployeeNo,Promoted_or_Not
0,YAK/S/00005,0
1,YAK/S/00011,0
2,YAK/S/00015,0
3,YAK/S/00016,0
4,YAK/S/00017,0
